In [11]:
!pip install picos numpy swiglpk ipykernel --quiet

In [12]:
import picos
import numpy as np

## Problema de inversión

In [13]:
# Crear el problema
P = picos.Problem()

# Variables binarias (una por inversion)
y = picos.BinaryVariable('y', 6)

# Funcion objetivo
P.set_objective('max', 15*y[0] + 12*y[1] + 16*y[2] + 18*y[3] + 9*y[4] + 11*y[5])

# Restricciones
P.add_constraint(38*y[0] + 33*y[1] + 39*y[2] + 45*y[3] + 23*y[4] + 27*y[5] <= 100)
P.add_constraint(y[0] + y[1] <= 1)  # inversiones 1 y 2 excluyentes
P.add_constraint(y[2] + y[3] <= 1)  # inversiones 3 y 4 excluyentes
P.add_constraint(y[2] <= y[0] + y[1])  # precedencia
P.add_constraint(y[3] <= y[0] + y[1])  # precedencia

# Resolver
P.options.verbosity = 0
P.solve(solver='glpk')

print(y)          # valores optimos de las variables
print(P.value)    # ganancia total optima

[ 1.00e+00]
[ 0.00e+00]
[ 1.00e+00]
[ 0.00e+00]
[ 1.00e+00]
[ 0.00e+00]
40.0


## Plantas y Productos

In [ ]:
import picos
import numpy as np

# ---- Datos ----
costos = np.array([
    [41, 27, 28, 24],
    [40, 29, 1e6, 23],  # 1e6 (1000000) es un costo prohibitivo para producto 3 en planta 2
    [37, 30, 27, 21]
])
capacidades = [75, 75, 45]
demanda = [20, 30, 30, 40]

plantas, productos = 3, 4

# ---- a) Permitir división (modelo de transporte) ----
P = picos.Problem()
x = picos.RealVariable('x', (plantas, productos), lower=0)

# Objetivo
P.set_objective('min', picos.sum([
    costos[i, j] * x[i, j]
    for i in range(plantas)
    for j in range(productos)
]))

# Restricciones de capacidad
for i in range(plantas):
    P.add_constraint(picos.sum(x[i, :]) <= capacidades[i])

# Restricciones de demanda
for j in range(productos):
    P.add_constraint(picos.sum(x[:, j]) == demanda[j])

P.solve(solver='glpk')
print('--- Modelo de transporte (división permitida) ---')
print("Asignación continua (x):")
print(np.round(x.value, 2))
print("Costo total:", round(P.value, 2))

# ---- b) No permitir división (modelo de asignación binaria) ----
costos_lista = costos.tolist()  # conversion segura para picos
demanda_lista = list(demanda)

P2 = picos.Problem()
y = picos.BinaryVariable('y', (plantas, productos))

# Objetivo
P2.set_objective('min', picos.sum([
    costos_lista[i][j] * demanda_lista[j] * y[i, j]
    for i in range(plantas)
    for j in range(productos)
]))

# Cada producto se fabrica en una sola planta
for j in range(productos):
    P2.add_constraint(picos.sum([y[i, j] for i in range(plantas)]) == 1)

# Restricciones de capacidad
for i in range(plantas):
    P2.add_constraint(picos.sum([
        y[i, j] * demanda_lista[j]
        for j in range(productos)
    ]) <= capacidades[i])

P2.solve(solver='glpk')
print('--- Modelo de asignación binaria (sin división) ---')
print("Asignación binaria (y):")
asignacion = np.array([[int(round(y[i, j].value)) for j in range(productos)] for i in range(plantas)])
print(asignacion)
print("Costo total:", round(P2.value, 2))

--- Modelo de transporte (división permitida) ---
Asignación continua (x):
[[ 0. 30. 30.  0.]
 [ 0.  0.  0. 15.]
 [20.  0.  0. 25.]]
Costo total: 3260.0
--- Modelo de asignación binaria (sin división) ---
Asignación binaria (y):
[[0 1 1 0]
 [1 0 0 0]
 [0 0 0 1]]
Costo total: 3290.0


## Estaciones de Bomberos

In [4]:
import picos
import numpy as np

# Matriz de tiempos de respuesta
tiempos = np.array([
    [5, 12, 30, 20, 15],
    [20, 4, 15, 10, 25],
    [15, 20, 6, 15, 12],
    [25, 15, 25, 4, 10],
    [10, 25, 15, 12, 5]
])

frecuencias = np.array([2, 1, 3, 1, 3])
sectores = 5

P = picos.Problem()

# Variables binarias
y = picos.BinaryVariable('y', sectores)  # y[i]: 1 si hay estacion en sector i
x = picos.BinaryVariable('x', (sectores, sectores))  # x[i,j]: 1 si sector j es atendido por estacion en i

# Objetivo: minimizar tiempo de respuesta ponderado por frecuencia
P.set_objective('min', picos.sum([
    tiempos[i, j] * x[i, j] * frecuencias[j]
    for i in range(sectores)
    for j in range(sectores)
]))

# Restriccion: cada sector es atendido por una sola estacion
for j in range(sectores):
    P.add_constraint(picos.sum([x[i, j] for i in range(sectores)]) == 1)

# Restriccion: solo puede atender si hay estacion en i
for i in range(sectores):
    for j in range(sectores):
        P.add_constraint(x[i, j] <= y[i])

# Restriccion: exactamente dos estaciones
P.add_constraint(picos.sum(y) == 2)

P.solve(solver='glpk')
print('Asignacion (x):')
print(np.array([[int(round(x[i, j].value)) for j in range(sectores)] for i in range(sectores)]))
print('Estaciones ubicadas (y):')
print(np.array([int(round(y[i].value)) for i in range(sectores)]))
print('Tiempo total ponderado:', round(P.value, 2))

Asignacion (x):
[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 1 1 0 0]
 [0 0 0 0 0]
 [1 0 0 1 1]]
Estaciones ubicadas (y):
[0 0 1 0 1]
Tiempo total ponderado: 85.0
